# Classificação de Discurso de Ódio 
-------------------------------------
## Abordagem: Bert com pos_weight na BCEWithLogitsLoss

-------------------------------------

## Importações e Definições

In [1]:
import pandas as pd
import numpy as np
from datasets import load_dataset
import torch

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset

from utils import preprocess_text, format_lime_output, print_multilabel_metrics

[nltk_data] Downloading package stopwords to /home/penido/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
target = ['aggressive', 'hate', 'ageism', 'aporophobia', 'body_shame', 'capacitism', 'lgbtphobia', 'political', 'racism', 'religious_intolerance', 'misogyny', 'xenophobia', 'other']
features = 'text'

In [3]:
# Configurações para o pré-processamento
config = {
    "lowercase": True,
    "remove_accents": True,
    "remove_punctuation": True,
    "remove_numbers": True,
    "remove_urls": True,
    "remove_mentions_hashtags": True,
    "expand_abbreviations": True,
    "expand_contractions": False,
    "normalize_laughter": True,
    "remove_emojis": True,
    "remove_stopwords": True,
    "lemmatize": True,
    "stemming": False,
    "pos_filter": False,
    "min_token_length": 2,
    "negation_scope": False,
    "replace_swears": False,
    "split_hashtags": False,
    "merge_mwes" : True,
    "replace_named_entities" : False
}

--------------------------------------------
## Prepara o Conjunto de Dados

In [4]:
# 1. Carrega o dataset TuPyE multilabel
df = load_dataset("Silly-Machine/TuPyE-Dataset", name="multilabel")

train_df = df['train'].to_pandas()
test_df = df['test'].to_pandas()

X_train_raw = train_df[features]
y_train = train_df[target].values

X_test_raw = test_df[features]
y_test = test_df[target].values

### Aplica Pré-Processamento

In [5]:
X_train = X_train_raw.apply(lambda x: preprocess_text(x, config))
X_test = X_test_raw.apply(lambda x: preprocess_text(x, config))

--------------------------------------------
## Treinamento do Modelo

In [6]:
# 2. Tokenizador BERT em português
model_name = "neuralmind/bert-base-portuguese-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)

In [7]:
# 3. Crie seu dataset personalizado
class MultilabelDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_length)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)

In [8]:
train_dataset = MultilabelDataset(X_train.tolist(), y_train, tokenizer)
test_dataset = MultilabelDataset(X_test.tolist(), y_test, tokenizer)

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
y_train_tensor = torch.tensor(y_train)  # ou use diretamente o tensor

# Contagem de positivos e negativos por classe
pos_counts = y_train_tensor.sum(dim=0)
neg_counts = y_train_tensor.shape[0] - pos_counts

# Peso positivo = (n negativos) / (n positivos)
pos_weight = neg_counts / (pos_counts + 1e-5)  # evita divisão por zero
pos_weight = torch.min(pos_weight, torch.tensor(45.0))
pos_weight = pos_weight.to(device)

In [11]:
pos_weight

tensor([ 2.8850,  7.3137, 45.0000, 45.0000, 45.0000, 45.0000, 45.0000, 36.9717,
        45.0000, 45.0000, 25.0701, 45.0000,  8.7093], device='cuda:0')

In [12]:
import torch.nn as nn

class MultilabelTrainer(Trainer):
    def __init__(self, *args, pos_weight=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.pos_weight = pos_weight
        self.loss_fn = nn.BCEWithLogitsLoss(pos_weight=self.pos_weight)

    def compute_loss(self, model, inputs, num_items_in_batch, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss = self.loss_fn(logits, labels.float())  # BCE precisa de float
        return (loss, outputs) if return_outputs else loss

In [13]:
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(target), problem_type="multi_label_classification")
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29794, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [14]:
training_args = TrainingArguments(
    output_dir='./results',       # Ainda obrigatório, mas o Trainer não vai salvar nada
    per_device_train_batch_size=10,
    per_device_eval_batch_size=8,
    num_train_epochs=8,
    logging_dir='./logs',
    logging_steps=50,
    save_strategy="no",           # <-- desativa o salvamento de modelos/checkpoints
)

In [15]:
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
# )

trainer = MultilabelTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    pos_weight=pos_weight 
)

In [16]:
trainer.train()

Step,Training Loss
50,0.906200
100,0.955900
150,0.835500
200,0.947000
250,0.748900
300,0.770700
350,0.756100
400,0.699100
450,0.790500
500,0.686400


TrainOutput(global_step=27952, training_loss=0.3349257654947046, metrics={'train_runtime': 13898.888, 'train_samples_per_second': 20.108, 'train_steps_per_second': 2.011, 'total_flos': 1.8384858809094144e+16, 'train_loss': 0.3349257654947046, 'epoch': 8.0})

-----------------------------------
## Avaliação no conjunto de Teste

In [17]:
def predict(text):
    model.eval()  # Garante que o modelo está em modo de avaliação
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Tokenização e envio dos inputs para o mesmo device do modelo
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.sigmoid(outputs.logits).cpu().numpy()[0]  # move o resultado de volta para CPU para conversão

    return dict(zip(target, probs))

In [18]:
# Exemplo:
print(predict("Esse cara é um idiota inútil"))

{'aggressive': 0.99023974, 'hate': 0.91881216, 'ageism': 0.00017316396, 'aporophobia': 6.9953436e-05, 'body_shame': 0.00043166854, 'capacitism': 0.0002922584, 'lgbtphobia': 0.00023030989, 'political': 0.002282296, 'racism': 0.0016543532, 'religious_intolerance': 0.00026386048, 'misogyny': 0.00082663837, 'xenophobia': 0.00045362307, 'other': 0.9559471}


In [19]:
# Exemplo:
print(predict("bom dia"))

{'aggressive': 0.0021874094, 'hate': 0.0005967587, 'ageism': 0.00019522557, 'aporophobia': 0.00015861474, 'body_shame': 0.0003246252, 'capacitism': 0.00027698535, 'lgbtphobia': 0.00050030154, 'political': 0.00022341401, 'racism': 0.0002879534, 'religious_intolerance': 0.0001736398, 'misogyny': 0.00057024206, 'xenophobia': 0.0003028833, 'other': 0.0023847884}


In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29794, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [21]:
from torch.utils.data import DataLoader

test_loader = DataLoader(test_dataset, batch_size=16)

all_probs = []
all_labels = []

In [22]:
from tqdm import tqdm

# 2. Inferência por batches no test set
with torch.no_grad():
    for batch in tqdm(test_loader, desc="Predicting"):
        inputs = {k: v.to(device) for k, v in batch.items() if k != "labels"}
        labels = batch["labels"].cpu().numpy()
        outputs = model(**inputs)
        probs = torch.sigmoid(outputs.logits).cpu().numpy()

        all_probs.append(probs)
        all_labels.append(labels)

# 3. Concatenar tudo
all_probs = np.concatenate(all_probs, axis=0)
all_labels = np.concatenate(all_labels, axis=0)

Predicting: 100%|██████████| 546/546 [02:47<00:00,  3.25it/s]


In [23]:
preds = (all_probs >= 0.5).astype(int)

In [24]:
print_multilabel_metrics(all_labels, preds)


📊 Avaliação Multilabel
✔️ F1 Score (Micro):     0.5871
✔️ F1 Score (Macro):     0.4401
✔️ F1 Score (Weighted):  0.5848
⚠️ Hamming Loss:         0.0363
✅ Subset Accuracy:      0.7299


In [25]:
# Considerando só agressive e hate
print_multilabel_metrics(all_labels[:, :2], preds[:, :2])


📊 Avaliação Multilabel
✔️ F1 Score (Micro):     0.6232
✔️ F1 Score (Macro):     0.5975
✔️ F1 Score (Weighted):  0.6205
⚠️ Hamming Loss:         0.1379
✅ Subset Accuracy:      0.7865
